In [87]:

from requests import get, post
from web3 import Web3, HTTPProvider
from icecream import ic
import numpy as np
import pandas as pd
import sys
from dotenv import load_dotenv
import os
import json
import matplotlib.pyplot as plt
from datetime import datetime
import time

In [88]:
# Load environment variables
load_dotenv()

# Get Etherscan API key from environment variables
ETHERSCAN_API_KEY = os.getenv("ETHERSCAN_KEY")
ALCHEMY_KEY = os.getenv("ALCHEMY_KEY_2")
INFURA_KEY = os.getenv("INFURA_KEY")

URL = f"https://eth-mainnet.g.alchemy.com/v2/{ALCHEMY_KEY}"

In [89]:

ABI = [{"inputs":[],"name":"getReservesList","outputs":[{"internalType":"address[]","name":"arg_0","type":"address[]"}],"stateMutability":"view","type":"function"},
       {"inputs":[{"internalType":"address","name":"asset","type":"address"}],"name":"getReserveData","outputs":[{"components":[{"components":[{"internalType":"uint256","name":"data","type":"uint256"}],"internalType":"struct DataTypes.ReserveConfigurationMap","name":"configuration","type":"tuple"},{"internalType":"uint128","name":"liquidityIndex","type":"uint128"},{"internalType":"uint128","name":"currentLiquidityRate","type":"uint128"},{"internalType":"uint128","name":"variableBorrowIndex","type":"uint128"},{"internalType":"uint128","name":"currentVariableBorrowRate","type":"uint128"},{"internalType":"uint128","name":"currentStableBorrowRate","type":"uint128"},{"internalType":"uint40","name":"lastUpdateTimestamp","type":"uint40"},{"internalType":"uint16","name":"id","type":"uint16"},{"internalType":"address","name":"aTokenAddress","type":"address"},{"internalType":"address","name":"stableDebtTokenAddress","type":"address"},{"internalType":"address","name":"variableDebtTokenAddress","type":"address"},{"internalType":"address","name":"interestRateStrategyAddress","type":"address"},{"internalType":"uint128","name":"accruedToTreasury","type":"uint128"},{"internalType":"uint128","name":"unbacked","type":"uint128"},{"internalType":"uint128","name":"isolationModeTotalDebt","type":"uint128"}],"internalType":"struct DataTypes.ReserveDataLegacy","name":"arg_0","type":"tuple"}],"stateMutability":"view","type":"function"},
       ]
POOL_ADDRESS = "0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2"
W3 = Web3(HTTPProvider(URL))

ABI_RESERVE = [{"inputs":[],"name":"getAllReservesTokens","outputs":[{"components":[{"internalType":"string","name":"symbol","type":"string"},{"internalType":"address","name":"tokenAddress","type":"address"}],"internalType":"struct IPoolDataProvider.TokenData[]","name":"","type":"tuple[]"}],"stateMutability":"view","type":"function"}]
DATA_PROVIDER = "0x7B4EB56E7CD4b454BA8ff71E4518426369a138a3"

In [90]:
def get_reserve_list(block_number=None):
    
    try:
        # Create contract instance
        contract = W3.eth.contract(address=Web3.to_checksum_address(DATA_PROVIDER), abi=ABI_RESERVE)
        
        # Get the contract function
        contract_function = getattr(contract.functions, "getAllReservesTokens")
        
        # Call the function with arguments and block identifier if provided
        if block_number is not None:
            data = contract_function().call(block_identifier=int(block_number))
        else:
            data = contract_function().call()
        
        return data
    
    except Exception as e:
        return f'Error querying smart contract: {e}'
    
    
def get_asset_data(asset, block_number=None):
    
    try:
        # Create contract instance
        contract = W3.eth.contract(address=Web3.to_checksum_address(POOL_ADDRESS), abi=ABI)
        
        # Get the contract function
        contract_function = getattr(contract.functions, "getReserveData")
        
        # Call the function with arguments and block identifier if provided
        if block_number is not None:
            data = contract_function(asset).call(block_identifier=int(block_number))
        else:
            data = contract_function(asset).call()
        
        return data
    
    except Exception as e:
        return f'Error querying smart contract: {e}'
    
# def get_data_from_pool(function_name, *args, block_number=None):
    
#     try:
#         # Create contract instance
#         contract = W3.eth.contract(address=Web3.to_checksum_address(POOL_ADDRESS), abi=ABI)
        
#         # Get the contract function
#         contract_function = getattr(contract.functions, function_name)
        
#         # Call the function with arguments and block identifier if provided
#         if block_number is not None:
#             data = contract_function(*args).call(block_identifier=int(block_number))
#         else:
#             data = contract_function(*args).call()
        
#         return data
    
#     except Exception as e:
#         return f'Error querying smart contract: {e}'

def decode_reserve_configuration(config_value):
    config = int(config_value)
    
    def get_bits(start, end):
        mask = (1 << (end - start + 1)) - 1
        return (config >> start) & mask
    
    return {
        "ltv": get_bits(0, 15)/10000,
        "liquidationThreshold": get_bits(16, 31)/10000,
        "liquidationBonus": get_bits(32, 47)/10000,
        "decimals": get_bits(48, 55),
        "reserveIsActive": bool(get_bits(56, 56)),
        "reserveIsFrozen": bool(get_bits(57, 57)),
        "borrowingEnabled": bool(get_bits(58, 58)),
        "stableRateBorrowingEnabled": bool(get_bits(59, 59)),
        "assetIsPaused": bool(get_bits(60, 60)),
        "borrowingInIsolationModeEnabled": bool(get_bits(61, 61)),
        "siloedBorrowingEnabled": bool(get_bits(62, 62)),
        "flashLoaningEnabled": bool(get_bits(63, 63)),
        "reserveFactor": get_bits(64, 79)/10000,
        "borrowCap": get_bits(80, 115),
        "supplyCap": get_bits(116, 151),
        "liquidationProtocolFee": get_bits(152, 167)/10000,
        "eModeCategoryId": get_bits(168, 175),
        "unbackedMintCap": get_bits(176, 211),
        "debtCeilingForIsolationMode": get_bits(212, 251),
        "virtualAccountingEnabled": bool(get_bits(252, 252))
    }
    
    
def get_current_price(token_address):
    """
    Fetches the current price of a token from the Coin Llama API.

    Args:
        token_address (str): The address of the token.

    Returns:
        Optional[float]: The current price of the token or None if an error occurs.
    """
    base_url = "https://coins.llama.fi/prices/current"
    url = f"{base_url}/ethereum:{token_address}"
    
    try:
        response = get(url)
        response.raise_for_status()  # Raise an exception for HTTP errors
        data = response.json()
        price_info = data.get('coins', {}).get(f'ethereum:{token_address}', {})
        return price_info.get('price')
    except Exception as e:
        print(f"Error fetching price for {token_address}: {e}")
        return None
    

def get_current_unix_timestamp() -> int:
    """
    Returns the current Unix timestamp as an integer.

    Returns:
        int: The current Unix timestamp (seconds since January 1, 1970).
    """
    return int(time.time())

In [91]:
asset_list = get_reserve_list()

In [92]:
data = []
unix_timestamp = get_current_unix_timestamp()

for asset in asset_list:
    data_dict = {}
    
    reserve_data = get_asset_data(asset[1])
    data_dict["timestamp"] = unix_timestamp
    data_dict["symbol"] = asset[0]
    data_dict["price"] = get_current_price(asset[1])
    data_dict["assetAddress"] = asset[1]
    data_dict["configuration"] = reserve_data[0][0]

    data_dict["aTokenAddress"] = reserve_data[8]
    data_dict["stableDebtTokenAddress"] = reserve_data[9]
    data_dict["variableDebtTokenAddress"] = reserve_data[10]
    
    decoded_configuration = decode_reserve_configuration(reserve_data[0][0])
    data_dict.update(decoded_configuration)
    data.append(data_dict)

print(json.dumps(data, indent=4))


[
    {
        "timestamp": 1727302495,
        "symbol": "WETH",
        "price": 2581.67,
        "assetAddress": "0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2",
        "configuration": 7237005577332262213973186942896404317902510153474528182451012586290352496498,
        "aTokenAddress": "0x4d5F47FA6A74757f35C14fD3a6Ef8E3C9BC514E8",
        "stableDebtTokenAddress": "0x102633152313C81cD80419b6EcF66d14Ad68949A",
        "variableDebtTokenAddress": "0xeA51d7853EEFb32b6ee06b1C12E6dcCA88Be0fFE",
        "ltv": 0.805,
        "liquidationThreshold": 0.83,
        "liquidationBonus": 1.05,
        "decimals": 18,
        "reserveIsActive": true,
        "reserveIsFrozen": false,
        "borrowingEnabled": true,
        "stableRateBorrowingEnabled": false,
        "assetIsPaused": false,
        "borrowingInIsolationModeEnabled": false,
        "siloedBorrowingEnabled": false,
        "flashLoaningEnabled": true,
        "reserveFactor": 0.15,
        "borrowCap": 1400000,
        "supplyC

In [94]:
df = pd.DataFrame(data)



### Save this dataframe to a csv
df.to_csv("aave_asset_data.csv", index=False)


In [95]:
ABI_BAL = [{"inputs":[{"internalType":"address[]","name":"addresses","type":"address[]"},{"internalType":"address","name":"tokenAddress","type":"address"}],"name":"checkBalances","outputs":[{"internalType":"uint256[]","name":"","type":"uint256[]"}],"stateMutability":"view","type":"function"}]
BAL_ADDRESS = "0x2a879c62119DcA3601e54990F712F1bBf026612c"
INFURA_URL = f"https://mainnet.infura.io/v3/{INFURA_KEY}"
W33 = Web3(HTTPProvider(INFURA_URL))

def get_user_balance(user_addrress_list, asset, block_number=None):
    
    try:
        # Create contract instance
        contract = W33.eth.contract(address=Web3.to_checksum_address(BAL_ADDRESS), abi=ABI_BAL)
        
        # Get the contract function
        contract_function = getattr(contract.functions, "checkBalances")
        
        # Call the function with arguments and block identifier if provided
        if block_number is not None:
            data = contract_function(user_addrress_list,asset).call(block_identifier=int(block_number))
        else:
            data = contract_function(user_addrress_list,asset).call()
        
        return data
    
    except Exception as e:
        return f'Error querying smart contract: {e}'


In [62]:
from dune_client.types import QueryParameter
from dune_client.client import DuneClient
from dune_client.query import QueryBase

# Explicitly specify the path to .env file using forward slashes
load_dotenv(dotenv_path='../.env')

dune = DuneClient(
    api_key=os.getenv('DUNE_API_KEY2'),
    base_url="https://api.dune.com",
    request_timeout=5000 # request will time out after 300 seconds
)

In [98]:
query = QueryBase(
    query_id=4101003,
    params=[],
)

users = dune.run_query_dataframe(query)
users["timestamp"] = unix_timestamp


### Save this dataframe to a csv
users.to_csv("users.csv", index=False)


# Create a list of user addresses
user_addresses = users['user'].tolist()

users_checksum = []
for user in user_addresses:
    users_checksum.append(Web3.to_checksum_address(user))



2024-09-25 18:19:37,162 INFO dune_client.api.base executing 4101003 on medium cluster
2024-09-25 18:19:37,702 INFO dune_client.api.base waiting for query execution 01J8NM9EVH7H7BWWB0AA5M67XP to complete: ExecutionState.PENDING (queue position: 1)
2024-09-25 18:19:38,830 INFO dune_client.api.base waiting for query execution 01J8NM9EVH7H7BWWB0AA5M67XP to complete: ExecutionState.EXECUTING
2024-09-25 18:19:39,958 INFO dune_client.api.base waiting for query execution 01J8NM9EVH7H7BWWB0AA5M67XP to complete: ExecutionState.EXECUTING
2024-09-25 18:19:41,089 INFO dune_client.api.base waiting for query execution 01J8NM9EVH7H7BWWB0AA5M67XP to complete: ExecutionState.EXECUTING
2024-09-25 18:19:42,216 INFO dune_client.api.base waiting for query execution 01J8NM9EVH7H7BWWB0AA5M67XP to complete: ExecutionState.EXECUTING
2024-09-25 18:19:43,342 INFO dune_client.api.base waiting for query execution 01J8NM9EVH7H7BWWB0AA5M67XP to complete: ExecutionState.EXECUTING
2024-09-25 18:19:44,469 INFO dune_clie

In [99]:
# data_test = data[0:1]

user_position = []
for user in users_checksum:
    user_position_dict = {"user": user}
    user_position_dict["timestamp"] = unix_timestamp
    user_position.append(user_position_dict)

for asset in data:
    decimals = asset["decimals"]
    
    atoken_symbol = f"a{asset['symbol']}"
    dtoken_symbol = f"d{asset['symbol']}"
    
    atoken_balance = get_user_balance(users_checksum, asset["aTokenAddress"])
    dtoken_balance = get_user_balance(users_checksum, asset["variableDebtTokenAddress"])
    
    for index, user_dict in enumerate(user_position):
        user_dict[atoken_symbol] = atoken_balance[index] / 10**decimals
        user_dict[dtoken_symbol] = dtoken_balance[index] / 10**decimals
        
# Convert this to a dataframe
df_user_position = pd.DataFrame(user_position)

# Save this dataframe to a csv
df_user_position.to_csv("user_position.csv", index=False)


In [ ]:
'''
  struct ReserveData {
    //stores the reserve configuration
    ReserveConfigurationMap configuration;
    //the liquidity index. Expressed in ray
    uint128 liquidityIndex;
    //the current supply rate. Expressed in ray
    uint128 currentLiquidityRate;
    //variable borrow index. Expressed in ray
    uint128 variableBorrowIndex;
    //the current variable borrow rate. Expressed in ray
    uint128 currentVariableBorrowRate;
    //the current stable borrow rate. Expressed in ray
    uint128 currentStableBorrowRate;
    //timestamp of last update
    uint40 lastUpdateTimestamp;
    //the id of the reserve. Represents the position in the list of the active reserves
    uint16 id;
    //timestamp until when liquidations are not allowed on the reserve, if set to past liquidations will be allowed
    uint40 liquidationGracePeriodUntil;
    //aToken address
    address aTokenAddress;
    //stableDebtToken address
    address stableDebtTokenAddress;
    //variableDebtToken address
    address variableDebtTokenAddress;
    //address of the interest rate strategy
    address interestRateStrategyAddress;
    //the current treasury balance, scaled
    uint128 accruedToTreasury;
    //the outstanding unbacked aTokens minted through the bridging feature
    uint128 unbacked;
    //the outstanding debt borrowed against this asset in isolation mode
    uint128 isolationModeTotalDebt;
    //the amount of underlying accounted for by the protocol
    uint128 virtualUnderlyingBalance;
  }
  
((7237005577332262213973186942896404317902510153474528182451012586290352496498,),
 1031659215120487790563166607,
 19627221359796060493882528,
 1052348817879507493709217434,
 26319678184915331852416053,
 0,
 1727291135,
 0,
 '0x4d5F47FA6A74757f35C14fD3a6Ef8E3C9BC514E8',
 '0x102633152313C81cD80419b6EcF66d14Ad68949A',
 '0xeA51d7853EEFb32b6ee06b1C12E6dcCA88Be0fFE',
 '0x847A3364Cc5fE389283bD821cfC8A477288D9e82',
 37387375566166645479,
 0,
 0)
  
  struct ReserveConfigurationMap {
    //bit 0-15: LTV
    //bit 16-31: Liq. threshold
    //bit 32-47: Liq. bonus
    //bit 48-55: Decimals
    //bit 56: reserve is active
    //bit 57: reserve is frozen
    //bit 58: borrowing is enabled
    //bit 59: stable rate borrowing enabled
    //bit 60: asset is paused
    //bit 61: borrowing in isolation mode is enabled
    //bit 62: siloed borrowing enabled
    //bit 63: flashloaning enabled
    //bit 64-79: reserve factor
    //bit 80-115: borrow cap in whole tokens, borrowCap == 0 => no cap
    //bit 116-151: supply cap in whole tokens, supplyCap == 0 => no cap
    //bit 152-167: liquidation protocol fee
    //bit 168-175: eMode category
    //bit 176-211: unbacked mint cap in whole tokens, unbackedMintCap == 0 => minting disabled
    //bit 212-251: debt ceiling for isolation mode with (ReserveConfiguration::DEBT_CEILING_DECIMALS) decimals
    //bit 252: virtual accounting is enabled for the reserve
    //bit 253-255 unused
'''
